### GIAI ĐOẠN 1: LẤY DỮ LIỆU TRÊN WEBSITE
---
- **Về công cụ**: Sử dụng Selenimum và BeautifulSoup để lấy dữ liệu tự động từ website.
- **Điều kiện tiên quyết**: Có cài đặt Chrome drivers và thư viện Selenium.
---

#### **1. Import các thư viện cần thiết cho crawling data:**

In [15]:
# Thư viện beautifulSoup:
from bs4 import BeautifulSoup

# Thư viện selenium và lấy ra webdriver:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Thư viện time để sử dụng các hàm liên quan đến thời gian
import time

# Thư viện xử lý mảng, dataframe: 
import pandas as pd

#### **2. Tiến hành các bước lấy dữ liệu:**

In [16]:
# Mở trình duyệt Chrome ở chế độ toàn màn hình:
browser = webdriver.Chrome()
browser.maximize_window()

In [17]:
# Mở trang chủ của trang cần lấy dữ liệu, ở đây là website Premierleague.com
browser.get('https://www.premierleague.com/players')

---

##### *Giai đoạn 1: Tìm hiểu website và xử lý lần vào đầu tiên*
- **Một số lưu ý** khi mới bắt đầu vào website:
  - Khi mở website trên lên, mới bắt đầu vào ta sẽ thấy một bảng thông báo là phải **Accept all cookies** thì ta mới <br> có thể thực hiện tiếp trên dữ liệu.
  - Có một điểm bất cập ở đây là khi mới bắt đầu vào trang, ta phải đợi nó tải hết một số dữ liệu nên sẽ bị trễ một khoảng thời gian. <br>Với thời gian thực thi của cell tìm và click vào button accept all cookies, nó luôn luôn báo lỗi nên chúng em phải dùng phương thức <br> **time.sleep(3)** để hoãn lại 3 giây chờ web tải hoàn chỉnh rồi sau đó mới có thể nhấn vào được nút.
  - 3 giây là thời gian chúng em quan sát được để website tải được hoàn chỉnh ở môi trường lý tưởng (băng thông mạnh,...).
- Lỗi trả về và cách sửa lỗi được tham khảo: [Stackoverflow](https://stackoverflow.com/questions/65120003/elementnotinteractableexception-using-selenium-in-python)
  
---

In [18]:
# Phương thức delay thời gian tải trang.
time.sleep(5)   

# button_Class: Kiểm tra trên website rồi lấy thủ công về.
button_Class = '//button[@class="_2hTJ5th4dIYlveipSEMYHH BfdVlAo_cgSVjDUegen0F js-accept-all-close"]'

# Tìm chỗ button_class đó rồi thực hiện click vào (Accept all cookies).
browser.find_element(By.XPATH, button_Class).click()

----
- **Kiểu tải dữ liệu của trang:**
  - Giống như Facebook hay Twitter, trang này sẽ tải gần như vô hạn khi tiếp tục kéo xuống dưới cuối.
  - Nhóm em thực hiện đoạn code liên tục kéo xuống trang dữ liệu cho đến khi nào nó tải mới, làm hoài như vậy cho đến khi trang đưa ra những dữ liệu cuối cùng.
- Kéo trang để lấy hết đoạn html đầy đủ chứa các tag href - chứa toàn bộ link các cầu thủ.
- Đoạn code kéo trang được tham khảo tại: [Python forum](https://python-forum.io/thread-20175.html)
----

In [19]:
# Thời gian đợi cuộn tải đữ liệu
SCROLL_PAUSE_TIME = 20

# Lấy kích thước trang ở cuối.
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Cuộn đến cuối trang.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Cài đặt thời gian để load trang khi nó mở rộng thêm dữ liệu.
    time.sleep(SCROLL_PAUSE_TIME)

    # Tính toán với chiều dài cuộn mới và so sánh với chiều cao trang ban đầu.
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

---

##### *Giai đoạn 2: Lấy từng chỉ số cầu thủ và đưa ra kết quả cuối cùng.*

- Sử dụng song song các thư viện Selenium và thư viện BeautifulSoup để thực hiện việc lấy dữ liệu website chứa thông tin từng cầu thủ.
- Vì là dạng tổ chức theo các trang nên việc lấy dữ liệu sẽ rất lâu (chạy 146 phút mới hoàn thành việc lấy hết dữ liệu).
---

In [20]:
# List chứa danh sách các liên kết đến thông tin cá nhân của một cấu thủ.
hrefLst = []

# Lấy toàn bộ source html trên trang. 
html_text = browser.page_source
soup = BeautifulSoup(html_text)

# Đóng trang web: 
browser.close()

# Danh sách các đường dẫn cần lấy và xử lý thông tin
player_link = soup.find_all('a', 'playerName')
for link in player_link: 
    hrefLst.append('https:' + link.get('href'))

In [28]:
# Làm việc trên danh sách cầu thủ: 
player_data = [] 

# Mở lại trình duyệt và tiến hành vào từng cầu thủ
browser = webdriver.Chrome()
browser.maximize_window()

# Vòng lặp duyệt từng link href để vào đượt trang thống kê chỉ số từng cầu thủ.
for index, link in enumerate(hrefLst):
    browser.get(link)

    # Lúc mới đầu vào sẽ có nút chấp nhận cookies, xử lý nó.
    time.sleep(3)
    if (index == 0):
        button_Class = '//button[@class="_2hTJ5th4dIYlveipSEMYHH BfdVlAo_cgSVjDUegen0F js-accept-all-close"]'
        browser.find_element(By.XPATH, button_Class).click()
        
    # Tìm kiếm thẻ chứa nút bấm để qua trang thống kê.
    # Vì có 2 thẻ button đó là cho Overview và Stats. Nhóm em chỉ quan tâm đến Stats nên chuyển nút qua đó.
    browser.find_elements(By.CLASS_NAME, "btn-tab")[1].click()

    time.sleep(2)
    html_text = browser.page_source
    soup = BeautifulSoup(html_text)
    
    # Tìm ra tất cả các thẻ như tên, các mục thống kê rồi lấy các nội dung từ đó.
    name = soup.find('div', 'name t-colour').get_text()
    topStat = soup.find_all('span', 'allStatContainer')

    # Đưa các nội dung vào mảng một chiều như một phần tử của mảng 2 chiều toàn bộ tập dữ liệu.
    one_player = [st.get_text() for st in topStat]
    one_player.insert(0, name)
    
    player_data.append(one_player)

browser.close() 

#### **3. Tiến hành đưa dữ liệu về file csv chung:**
- Sử dụng thư viện Pandas để đưa mảng chứa các dữ liệu cầu thủ về loại DataFrames của Pandas.
- Sau khi chuyển về xong, ta viết các thông tin của Dataframe đó về một file CSV và đánh các tên chỉ số cho các cột dựa trên các thuộc tính lấy được.
- Viết lại vào một file csv để tiện dùng cho sau này.

In [39]:
dfPlayer = pd.DataFrame(player_data)
dfPlayer.columns = ['Player name', 'Appearances', 'Goals', 'Wins', 'Losses', 'Goals', 'Goals per match',
                    'Headed goals', 'Goals with right foot', 'Goals with left foot',
                    'Penalties scored', 'Freekicks scored', 'Shots', 'Shots on target', 
                    'Shooting accuracy %', 'Hit woodwork', 'Big chances missed', 'Assists',
                    'Passes', 'Passes per match', 'Big chances created', 'Crosses', 'Cross accuracy %',
                    'Through balls', 'Accurate long balls', 'Yellow cards', 'Red cards',
                    'Fouls', 'Offsides', 'Tackles', 'Tackle success %', 'Blocked shots',
                    'Interceptions', 'Clearances', 'Headed clearances', 'Recoveries', 'Duels won',
                    'Duels lost', 'Succesful 50/50s', 'Aerial battles won', 'Aerial battles lost', 
                    'Errors leading to goal']

dfPlayer.to_csv('player_data.csv', index=False)

In [42]:
# Hiện sơ lược kết quả đã thu về:
dfPlayer

,Player name,Appearances,Goals,Wins,Losses,Goals,Goals per match,Headed goals,Goals with right foot,Goals with left foot,Penalties scored,Freekicks scored,Shots,Shots on target,Shooting accuracy %,Hit woodwork,Big chances missed,Assists,Passes,Passes per match,Big chances created,Crosses,Cross accuracy %,Through balls,Accurate long balls,Yellow cards,Red cards,Fouls,Offsides,Tackles,Tackle success %,Blocked shots,Interceptions,Clearances,Headed clearances,Recoveries,Duels won,Duels lost,Succesful 50/50s,Aerial battles won,Aerial battles lost,Errors leading to goal
0,Brenden Aaronson,14,1,4,7,1,0.07,0,1,0,0,0,19,4,21%,2,1,2,400,28.57,5,42,26%,3,5,2,0,8,3,24,25%,5,1,5,1,68,77,106,19,6,17,0
1,Zach Abbott,0,0,0,0,0,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,None,None
2,Terry Ablade,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0%,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0,0,None,None,None,None,None,None,None,None,None,None,None
3,Tammy Abraham,89,26,36,33,26,0.29,3,20,1,0,0,164,68,41%,3,30,5,922,10.36,7,32,2,0,56,39,25,31,12,55,45,None,None,None,None,None,None,None,None,None,None,None
4,Adam Armstrong,52,3,8,32,3,0.06,0,1,2,0,0,71,23,32%,0,5,4,570,10.96,4,39,2,0,28,4,24,19,6,10,7,None,None,None,None,None,None,None,None,None,None,None
5,Adama Traoré,172,9,49,76,9,0.05,1,7,1,0,0,149,41,28%,6,9,14,"2,551",14.83,23,547,8,0,142,30,122,55,34,40,22,None,None,None,None,None,None,None,None,None,None,None
6,Tyler Adams,13,0,4,6,0,0.00,0,0,0,0,0,2,0,0%,0,0,0,742,57.08,1,2,0%,1,23,2,1,25,0,52,54%,1,16,11,2,104,75,62,2,10,7,0
7,Tosin Adarabioyo,43,1,9,19,10,61,48,48%,0,3,53,241,121,0,183,172,99,12,105,44,1,1,0,"2,384",55.44,0,3,33%,0,142,1,0,22,2,1,1,0,0,0,None,None
8,Tayo Adaramola,0,0,0,0,0,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,None,None
9,Joshua Addae,0,0,0,0,0,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0%,0,0,0,0,0,0,0,0,0,0,0,None,None


---
#### 4. Đánh giá và báo cáo giai đoạn crawling data:
- Đánh giá bước lấy dữ liệu: 
  - **Hạn chế**: 
    - Việc thực thi các cell lấy dữ liệu mất nhiều thời gian (146 phút) để có thể lấy được toàn bộ dữ liệu. Nguyên nhân một phần là vì các thao tác như sang trang cầu thủ, bấm vào nút thống kê để xem chi tiết, lấy các dữ liệu cần thiết... Và cũng một phần là phải thực hiện kéo trang đến cuối để có thể hiện được hết thông tin.
    - Thay đổi trang web vì phải tìm được trang ít lỗi và tĩnh nhất có thể.
    - Có một vài trang dữ liệu có định dạng khác format thông thường vì các cầu thủ đó chưa đá nên chưa có dữ liệu, bắt buộc ở bước tiền xử lý dữ liệu thực hiện sau.
    - Giai đoạn hoàn chỉnnh mất nhiều tuần.
  - **Thuận lợi:**
    - Không.
  